# Machine learning

In [ ]:
from transformers import (
    AutoModelForSequenceClassification, 
    AutoTokenizer
)
import transformers
import numpy as np
import torch
import random
import utils


In [15]:
device = 'cpu'
num_labels = 36
tokenizer_name = 'xlm-roberta-base'

In [16]:
model = AutoModelForSequenceClassification.from_pretrained('model', num_labels=num_labels).to(device)
config = model.config
max_model_length = config.max_position_embeddings
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

In [8]:
def convert_logit(logit):
    res = []
    for i in range(0, 36, 6):
        x = logit[i:i+6]
        res.append(torch.argmax(x))
    res = torch.stack(res)
    return res

In [9]:
review_sentence = 'không gian hiện đại phục vu chu đáo'
input = tokenizer(review_sentence, return_tensors="pt", padding='max_length', truncation=True, max_length=64).to(device)
logit = model(**input)[0][0]
predict_results = convert_logit(logit).tolist()
print(predict_results)
RATING_ASPECTS = ["giai_tri", "luu_tru", "nha_hang", "an_uong", "di_chuyen", "mua_sam"]
output = {
        "review": review_sentence,
        "results": {}
      }
for count, r in enumerate(RATING_ASPECTS):
    output["results"][r] = predict_results[count]
print(output)

[0, 5, 0, 0, 0, 0]
{'review': 'không gian hiện đại phục vu chu đáo', 'results': {'giai_tri': 0, 'luu_tru': 5, 'nha_hang': 0, 'an_uong': 0, 'di_chuyen': 0, 'mua_sam': 0}}


# Spark

In [2]:
import findspark
from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext
from pyspark.sql.functions import desc
import sys
import requests
import re

In [3]:
findspark.init()

In [4]:
# create spark configuration
conf = SparkConf()
conf.setAppName("ReviewStreamApp")
# create spark instance with the above configuration
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
# creat the Streaming Context from the above spark context with window size 2 seconds
ssc = StreamingContext(sc, 10)
# setting a checkpoint to allow RDD recovery
ssc.checkpoint("checkpoint_ReviewApp")
# # read data from port 9009
# dataStream = ssc.socketTextStream("localhost",9009)
# ssc = StreamingContext(sc, 10)
sqlContext = SQLContext(sc)

socket_stream = ssc.socketTextStream("127.0.0.1", 1606)

lines = socket_stream.window(20)
lines.pprint()

22/12/21 16:27:43 WARN Utils: Your hostname, ThinkPad-W540 resolves to a loopback address: 127.0.1.1; using 192.168.48.120 instead (on interface wlp3s0)
22/12/21 16:27:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/21 16:27:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


/home/caohainam/anaconda3/envs/add-std/lib/python3.8/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [4]:
# type(dataStream)

In [6]:
# dataStream.pprint()

In [6]:
# dataStream.count().pprint()

In [7]:
import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
import pandas
# Only works for Jupyter Notebooks!

In [1]:
# ssc.start()

In [ ]:
# count = 0
# while count < 10:
    
#     time.sleep( 3 )
#     top_10_tweets = sqlContext.sql( 'Select tag, count from tweets' )
#     top_10_df = top_10_tweets.toPandas()
#     display.clear_output(wait=True)
#     plt.figure( figsize = ( 10, 8 ) )
#     sns.barplot( x="count", y="tag", data=top_10_df)
#     plt.show()
#     count = count + 1

In [2]:
import pandas as pd 
from config import *
import re
import utils

In [10]:
RATING_ASPECTS = ["giai_tri", "luu_tru", "nha_hang", "an_uong", "di_chuyen", "mua_sam"]

In [9]:
def get_sentiment(score):
    if score >= 4:
        return 'POSITIVE'
    elif score == 3:
        return 'NORMAL'
    else:
        return 'NEGATIVE'

In [19]:
print('\nReading file...\n')
data = pd.read_csv(DATA_PATH)
for index, row in data.iterrows():
    review = row['review']
    input = tokenizer(review, return_tensors="pt", padding='max_length', truncation=True, max_length=64).to(device)
    logit = model(**input)[0][0]
    predict_results = utils.convert_logit(logit).tolist()
    lines = []
    
    for count, r in enumerate(RATING_ASPECTS):
        if predict_results[count] > 0:
            lines.append([r, utils.get_sentiment(predict_results[count])])
    print(lines)
    break


Reading file...

[['giai_tri', 'POSITIVE']]


In [ ]:
aspects = ['giai_tri', 'luu_tru', 'nha_hang', 'an_uong', 'van_chuyen', 'mua_sam']

In [8]:
for index, row in data.iterrows():
    review = row['review']
    print(review)
    break

Sân mới được nâng cấp, sạch và mới, đặc biệt là mặt sân. Phần chỗ ngồi đa phần không có ghế, ngồi bệ xi măng.


In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]") \
    .appName("SparkByExamples.com").getOrCreate()

data = ["Project","Gutenberg’s","Alice’s","Adventures",
"in","Wonderland","Project","Gutenberg’s","Adventures",
"in","Wonderland","Project","Gutenberg’s"]

rdd=spark.sparkContext.parallelize(data)
type(rdd)

pyspark.rdd.RDD

In [8]:
from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext
import sys
import requests
import re

# create spark configuration
conf = SparkConf()
conf.setAppName("ReviewStreamApp")
# create spark instance with the above configuration
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
# creat the Streaming Context from the above spark context with window size 2 seconds
ssc = StreamingContext(sc, 2)
# setting a checkpoint to allow RDD recovery
ssc.checkpoint("checkpoint_ReviewApp")
# read data from port 9009
dataStream = ssc.socketTextStream("localhost",1606)

22/12/22 09:35:46 WARN Utils: Your hostname, ThinkPad-W540 resolves to a loopback address: 127.0.1.1; using 192.168.48.120 instead (on interface wlp3s0)
22/12/22 09:35:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/22 09:35:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
type(dataStream)

pyspark.streaming.dstream.DStream

In [10]:
def aggregate_tags_count(new_values, total_sum):
    total_sum = total_sum if total_sum else (0,0,0,0)
    pos = [field[0] for field in new_values]
    neu = [field[1] for field in new_values]
    neg = [field[2] for field in new_values]
    count = [field[3] for field in new_values]

    return sum(pos)+total_sum[0], sum(neu)+total_sum[1], sum(neg)+total_sum[2], sum(count)+total_sum[3]

def print_data(df):
    # extract the hashtags from dataframe and convert them into array
    aspects = [str(t.aspect) for t in df.select("aspect").collect()]
    # extract the counts from dataframe and convert them into array
    pos = [p.pos for p in df.select("pos").collect()]
    neu = [p.neu for p in df.select("neu").collect()]
    neg = [p.neg for p in df.select("neg").collect()]
    request_data = {'label': str(aspects), 'data_pos': str(pos), 'data_neu': str(neu), 'data_neg': str(neg)}
    print(request_data)

def get_sql_context_instance(spark_context):
    if ('sqlContextSingletonInstance' not in globals()):
        globals()['sqlContextSingletonInstance'] = SQLContext(spark_context)
    return globals()['sqlContextSingletonInstance']

def process_rdd(time, rdd):
    print("----------- %s -----------" % str(time))
    try:
        # Get spark sql singleton context from the current context
        sql_context = get_sql_context_instance(rdd.context)
        # convert the RDD to Row RDD
        # row_rdd = rdd.map(lambda w: Row(aspect=w[0].encode("utf-8"), pos=w[1][0], neu=w[1][1], neg=w[1][2],  aspect_count=w[1][3]))
        row_rdd = rdd.map(lambda w: Row(aspect=w[0], pos=w[1][0], neu=w[1][1], neg=w[1][2],  aspect_count=w[1][3]))
        # create a DF from the Row RDD
        aspects_df = sql_context.createDataFrame(row_rdd)
        # Register the dataframe as table
        aspects_df.registerTempTable("aspects")
        # get the top 10 hashtags from the table using SQL and print them
        aspect_counts_df = sql_context.sql("select aspect, aspect_count, pos, neu, neg from aspects order by aspect_count desc")
        aspect_counts_df.show()
        # call this method to prepare top 10 hashtags DF and send them
        # send_df_to_dashboard(aspect_counts_df)
        print_data(aspect_counts_df)
    except:
        e = sys.exc_info()[0]
        print("Error: %s" % e)

In [11]:
def split_word(line):
    # return line
    data = line.split("||||")
    aspect = data[0]
    if data[1] == "POSITIVE":
        result = (aspect, 1, 0, 0)
    elif data[1] == "NORMAL":
        result = (aspect, 0, 1, 0)
    else:
        result = (aspect, 0, 0, 1)

    return result

In [12]:
# words = dataStream.flatMap(split_word)

In [13]:
# type(words)

In [14]:
# print(dir(words))

In [15]:
# words.pprint()
# dataStream.pprint()
# words.context()

In [16]:
stream_data = dataStream.map(lambda x: split_word(x))
stream_data = stream_data.map(lambda x: (x[0], (x[1], x[2], x[3], 1)))
# adding the count of each hashtag to its last count
stream_data = stream_data.updateStateByKey(aggregate_tags_count)
# do processing for each RDD generated in each interval
stream_data.foreachRDD(process_rdd)

In [17]:
# stream_data.pprint()

In [2]:
# ssc.start()
# ssc.awaitTermination()

# Database MySQL

In [37]:
import mysql.connector
from datetime import datetime
import utils

In [38]:
params = utils.config_db_params()
connection = mysql.connector.connect(**params)
mycursor = connection.cursor()

In [39]:
params 

{'host': 'localhost',
 'database': 'tourism_review',
 'user': 'namch',
 'password': 'Namch@1234'}

## Insert to db

In [44]:
def checkTableExists(dbcon, tablename):
    dbcur = dbcon.cursor()
    dbcur.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(tablename.replace('\'', '\'\'')))
    if dbcur.fetchone()[0] == 1:
        dbcur.close()
        return True

    dbcur.close()
    return False

In [45]:
checkTableExists(connection, 'customers')

True

In [27]:
mycursor = connection.cursor()

In [10]:
mycursor.execute("CREATE TABLE review_1 (review VARCHAR(255), giai_tri INT, luu_tru INT, nha_hang INT, an_uong INT, di_chuyen INT, mua_sam INT, time VARCHAR(20))")

In [30]:
sql = "INSERT INTO review_1 (review, giai_tri, luu_tru, nha_hang, an_uong, di_chuyen, mua_sam, time) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
now = datetime.now()
time = now.strftime('%Y-%m-%d %H:%M:%S')
val = ("John", 1, 1, 1, 1, 1, 1, time)
mycursor.execute(sql, val)
connection.commit()

In [4]:
datetime.now()

datetime.datetime(2022, 12, 27, 22, 6, 58, 909713)

In [14]:
from configparser import ConfigParser

def config(filename='database.ini', section='mysql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db

In [15]:
params = config()

In [16]:
params

{'host': 'localhost',
 'database': 'tourism_review',
 'user': 'namch',
 'password': 'Namch@1234'}

## Query db

In [35]:
start_time, end_time = '2022-12-26', '2022-12-29'
# query = f"SELECT * FROM review where time > {start_time}"
# query = f'select * from review where giai_tri=5'
query = f"select * from review where time between {start_time} and {end_time}"
mycursor.execute(query)

In [36]:
mycursor.fetchall()

[]